# Universal Pleasantness: Bayesian Modeling

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from copy import copy
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import pathlib
import seaborn as sns
from tqdm.auto import tqdm
sns.set(font_scale=1.2)
import univple as up

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load the data and show the ranking for each odorant

In [3]:
data = {}
data['raw'], odorants = up.load_data()
groups, group_ids = up.get_groups(data['raw'])
data['raw'].head()  # First 5 rows

OdorName               1-Octen-3-ol  2-Isobutyl-3-methoxypyrazine  \
Group     Participant                                               
Cha'palaa 1                      10                             6   
          2                       5                             4   
          3                       7                             8   
          4                       7                            10   
          5                       6                            10   

OdorName               2-Phenylethanol  Diethyl disulfide  Ethyl butyrate  \
Group     Participant                                                       
Cha'palaa 1                          2                  8               4   
          2                          2                  3               8   
          3                          1                 10               5   
          4                          3                  4               9   
          5                          2                  8               5   

OdorName               Eugenol  Isovaleric acid  Linalool  Octanoic acid  \
Group     Participant                                                      
Cha'palaa 1                  3                7         1              9   
          2                  1                6         9             10   
          3                  4                3         6              9   
          4                  1                8         6              5   
          5                  3                7         1              9   

OdorName               Vanillin  
Group     Participant            
Cha'palaa 1                   5  
          2                   7  
          3                   2  
          4                   2  
          5                   4

### For modeling purposes, switch this to the odor index for rank 1, for rank 2, etc. for each subject

In [4]:
data['ranked'], odorants = up.load_data(by='ranks')
groups, group_ids = up.get_groups(data['ranked'])
data['ranked'].head()  # First 5 rows

1st  2nd  3rd  4th  5th  6th  7th  8th  9th  10th
Group     Participant                                                   
Cha'palaa 1              8    3    6    5   10    2    7    4    9     1
          2              6    3    4    2    1    7   10    5    8     9
          3              3   10    7    6    5    8    1    2    9     4
          4              6   10    3    4    9    8    1    7    5     2
          5              8    3    6   10    5    1    7    4    9     2

### A Bayesian Model for Conducting the Same Analysis

In [5]:
# Compile the model, written in Stan (a probabilistic programming language for fitting Bayesian models)
model = up.load_or_compile_stan_model('univple')

In [ ]:
# Create shuffles to be modeled as well.
shuffles = ['odors-within-culture', 'individuals', 'odors']
for shuffle in shuffles:
    data['shuffle-%s' % shuffle] = up.shuffle_data(data['ranked'], groups, shuffle)

# Fit the model to the real data and to the shuffled data
samples = {}
for shuffle in tqdm(['ranked'] + ['shuffle-%s' % shuffle for shuffle in shuffles]):
    %time samples[shuffle] = up.load_or_sample(model, data[shuffle], shuffle)


Gradient evaluation took 0.000629 seconds
1000 transitions using 10 leapfrog steps per transition would take 6.29 seconds.
Adjust your expectations accordingly!


Iteration:     1 / 20000 [  0%]  (Warmup)


Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in 'univple.stan' at line 52)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.




Gradient evaluation took 0.000673 seconds
1000 transitions using 10 leapfrog steps per transition would take 6.73 seconds.
Adjust your expectations accordingly!



Gradient evaluation took 0.000792 seconds
1000 transitions using 10 leapfrog steps per transition would take 7.92 seconds.
Adjust your expectations accordingly!



Gradient evaluation took 0.000643 seconds
1000 transitions using 10 leapfrog steps per transition would take 6.43 seconds.
Adjust your expectations accordingly!


Iteration:     1 / 20000 [  0%]  (Warmup)
Iteration:     1 / 20000 [  0%]  (Warmup)


Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in 'univple.stan' at line 52)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.



Iteration:     1 / 20000 [  0%]  (Warmup)


Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in 'univple.stan' at line 52)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in 'univple.stan' at line 48)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.

Informational Message: The current Metropolis proposal is about to be rejected because of the following 

Iteration:  2000 / 20000 [ 10%]  (Warmup)
Iteration:  2000 / 20000 [ 10%]  (Warmup)
Iteration:  2000 / 20000 [ 10%]  (Warmup)
Iteration:  2000 / 20000 [ 10%]  (Warmup)
Iteration:  4000 / 20000 [ 20%]  (Warmup)
Iteration:  4000 / 20000 [ 20%]  (Warmup)
Iteration:  4000 / 20000 [ 20%]  (Warmup)
Iteration:  5001 / 20000 [ 25%]  (Sampling)
Iteration:  4000 / 20000 [ 20%]  (Warmup)
Iteration:  5001 / 20000 [ 25%]  (Sampling)
Iteration:  5001 / 20000 [ 25%]  (Sampling)
Iteration:  5001 / 20000 [ 25%]  (Sampling)
Iteration:  7000 / 20000 [ 35%]  (Sampling)
Iteration:  9000 / 20000 [ 45%]  (Sampling)
Iteration:  7000 / 20000 [ 35%]  (Sampling)
Iteration:  7000 / 20000 [ 35%]  (Sampling)
Iteration: 11000 / 20000 [ 55%]  (Sampling)
Iteration:  7000 / 20000 [ 35%]  (Sampling)
Iteration: 13000 / 20000 [ 65%]  (Sampling)
Iteration:  9000 / 20000 [ 45%]  (Sampling)
Iteration:  9000 / 20000 [ 45%]  (Sampling)
Iteration: 15000 / 20000 [ 75%]  (Sampling)
Iteration: 11000 / 20000 [ 55%]  (Sampling)
Iter

In [ ]:
# Check if independent sampling chains (runs with different random starting points) agree on global odorant valences
# If each odorant produces 4 almost perfectly overlapping histograms,
# then the fit went well
up.plot_global_agreement(samples, odorants)

### Model Consistency Check: Estimated valences and observed ranks are almost perfectly anti-correlated at the individual level (i.e. a high valence is a low (e.g. 1) rank.

In [ ]:
# Distribution of individual-level Pearson Correlation between observed rankings
# and estimated latent valences.  Should be close to -1
# (since lower ranking is more pleasant, but higher rating is more pleasant).
# Differs from exactly -1 due to a) the switch between a cardinal and ordinal measure
# and b) the denoising applied by the model
up.ranks_vs_values(samples, groups)

### Show how the group mean valences for each odorant compare.  Order by global mean valence.

In [ ]:
up.plot_global_means(groups, odorants, samples)

### Estimate contributions of each factor using the Bayesian model.

In [ ]:
up.plot_sigmas(samples, groups)

### Verify that when the model is applied to the shuffled ranks, the contribution of global valence disappears.

In [ ]:
up.plot_sigmas(samples_group_shuffle, groups)

### Valences are very highly correlated across groups (different groups ranks things similarly), but odorants are not really correlated with one another (how a group ranks one odorant does not closely predict how it will rank another odorant)

In [ ]:
up.get_means(samples, groups, odorants, 'mu_group')

In [ ]:
up.corr_heatmaps(samples, groups, odorants)

### For the shuffle control, this correlation structure (among groups) disappears

In [ ]:
up.corr_heatmaps(samples_group_shuffle, groups, odorants)

### PCA shows that there is really only one principal component explaining most of the variance in cross-cultural odor ranking.  Operating in the other direction, on odorants, shows a higher-dimensional structure (recapitulates the point in the figures above).

In [ ]:
up.plot_var_explained(samples, groups, odorants)

### The model estimates the intra-group standard deviations

In [ ]:
# Individual variability within a group
intragroup_std = pd.Series([samples['sigma_ind[%d]' % x].mean() for x in range(1, len(groups)+1)], index=groups)
intragroup_std.sort_values().to_frame(name='Standard Deviation across Individuals').round(1)

### We can examine this more closely for each group, looking at the correlation in estimated odorant valences between individuals.  This shows that the Maniq are ranking in a way which is indistinguishable from random (red dashed line, KS-test p-value shown)

In [ ]:
up.plot_ind_corrs(data, samples, groups, odorants)

### We can look at the groups in a reduced dimension using PCA (big dots are group means, little dots are individuals; many individuals are far away from the mean, illustrating that individual variability (and noise) is large compared to the differences between groups.

In [ ]:
up.plot_all_individuals(samples, groups, group_ids, odorants)

### We can then partition the groups into supergroups and ask if these supergroups explain the observed group structure

In [ ]:
supergroups = {'subsistence': {'Industrial': ['New York City', 'Thai', 'Mexican'],
                               'Agricultural': ['Semelai', 'Mah Meri', "Cha'palaa", 'Imbabura Quechua'],
                               'Hunter-gatherer': ['Maniq', 'Semaq Beri', 'Seri']},
               'geography': {'North America': ['New York City', 'Seri', 'Mexican'],
                             'South America': ["Cha'palaa", 'Imbabura Quechua'],
                              'Asia': ['Semelai', 'Mah Meri', 'Maniq', 'Semaq Beri', 'Thai']}}

In [ ]:
# Plot using PCA as above
up.plot_supergroups(samples, groups, odorants, supergroups, method='PCA')

In [ ]:
# Plot using Multidimensional Scaling, another dimensionality reduction strategy
up.plot_supergroups(samples, groups, odorants, supergroups, method='MDS')

### We next ask whether certain supergroups summarize the group structure.  We compare these groups to random group assignments of the same size to get p-values

In [ ]:
supergroup_stats = up.get_supergroup_stats(samples, groups, odorants, supergroups)
supergroup_stats.to_frame(name='p')